In [1]:
import torch
import time
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

In [2]:
def init_model():
    model = AutoModelForCausalLM.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        torch_dtype=torch.float16,
        device_map="cuda:1",
        trust_remote_code=True
    )
    model.generation_config = GenerationConfig.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat"
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        use_fast=False,
        trust_remote_code=True
    )
    return model, tokenizer
model, tokenizer = init_model()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def init_model_withParamater():
    model = AutoModelForCausalLM.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        torch_dtype=torch.float16,
        device_map="cuda:1",
        trust_remote_code=True,
    )
    model.generation_config = GenerationConfig.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        do_sample=True, 
        top_p=8, 
        temperature=1.2,
        repetition_penalty = 1.1,
        num_beams = 2,
        early_stopping = True,
        no_repeat_ngram_size=3
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "/data/tds/Baichuan2/Baichuan2-13B-Chat",
        use_fast=False,
        trust_remote_code=True
    )
    return model, tokenizer
model_withParamater, tokenizer_withParamater = init_model_withParamater()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
df = pd.read_csv("/home/kit/kit/ChatGLM3/langchain_demo/example_data.csv",sep=',')
df = df.drop(columns='Unnamed: 0', axis=1) 
df = df.astype(str)
df.replace('nan', ' ', inplace=True)   
df = df.assign(generate_sentence_top3 = None,generate_sentence_top2 = None,generate_sentence_top3 = None,generate_sentence_withParamater_top3 = None,generate_sentence_withParamater_top2 = None,generate_sentence_withParamater_top3 = None)

In [7]:
def create_prompt(prompt,example_sentence,df,i):
    s = prompt.replace(prompt[729:780],example_sentence)
    df.iloc[i,10] = str(df.iloc[i,10])
    s = s[:181] + df.iloc[i,0] + s[200:203] + df.iloc[i,1] + s[205:231] +  df.iloc[i,2] + s[383:408] +  df.iloc[i,4] + s[409:434] +  df.iloc[i,3] + s[437:462] +  df.iloc[i,5] + s[463:488]+  df.iloc[i,6] + s[489:514]+  df.iloc[i,7] + s[514:539]+  df.iloc[i,8] + s[539:565]+  df.iloc[i,9] + s[565:591]+  df.iloc[i,10] + s[591:672] + df.iloc[i,11] + s[672:]  
    return s

In [16]:
def get_response_print(model,tokenizer,prompt):
    messages = []
    messages.append({"role": "user", "content": prompt})
    response = model.chat(tokenizer, messages)
    return response

In [17]:
prompt = """
你是一位精通营销知识和文案策划的小红书爆款写作专家，您的任务通过学习样例文案的语言风格，创作出满足以下要求的小红书文风内容，你不允许回答“好的，当然可以”等描述性词语，也不允许回答“标题，正文”等格式性内容，请勿直接照抄样例文案，你只需要将创作好的内容返回给用户即可，
               要求：  
                   主题：推荐卡尔文·克莱恩Calvin Klein品牌的内衣
                   品牌卖点： 夏款时尚配色、缤纷色彩、花样繁多、超细纤维面料、舒适透气、金典款式、简约大方、高级感、CK、CK内衣、CK男士内裤、CK男士平角裤、男士礼物、男士内裤推荐、超级舒适的男士内裤、夏季男士内裤、夏季冰丝内裤、CK代言人田柾国、CK代言人JENNIE、BLACKPINK、BTS防弹少年团、CK段宜恩、CK主打
                   受众群体： 
                   平台文风：小红书
                   使用场景： 
                   推荐角度： 
                   文案风格： 
                   故事性： 
                   字数要求： 
                   营销目的： 
                   写作要求：使用惊叹号、表情符号等增加文章活力，运用悬念来引发读者好奇心
                   其他要求： 
               样例文案：
               ```
               夏款时尚配色，缤纷色彩🍃，花样繁多 CK主打💯超细纤维面料，舒适透气✅ 金典款式，简约大方，高级感拉🈵
               ```
"""
for i in range(int(len(df))):
    s_top1 = create_prompt(prompt,df.iloc[i,12],df,i)
    s_top2 = create_prompt(prompt,df.iloc[i,13],df,i)
    s_top3 = create_prompt(prompt,df.iloc[i,14],df,i)

    response_top1 = get_response_print(model,tokenizer,s_top1)
    response_top2 = get_response_print(model,tokenizer,s_top2)
    response_top3 = get_response_print(model,tokenizer,s_top3)

    response_withParamater_top1 = get_response_print(model_withParamater,tokenizer_withParamater,s_top1)
    response_withParamater_top2 = get_response_print(model_withParamater,tokenizer_withParamater,s_top2)
    response_withParamater_top3 = get_response_print(model_withParamater,tokenizer_withParamater,s_top3)
    
    print(i,s_top1)
    print("response_top1: " + response_top1)
    print("response_withParamater_top1: " + response_withParamater_top1)
    df.iloc[i,15] = response_top1
    df.iloc[i,18] = response_withParamater_top1

    print(i,s_top2)
    print("response_top2: " + response_top2)
    print("response_withParamater_top2: " + response_withParamater_top2)
    df.iloc[i,16] = response_top2
    df.iloc[i,19] = response_withParamater_top2

    print(i,s_top3)
    print("response_top3: " + response_top3)
    print("response_withParamater_top3: " + response_withParamater_top3)
    df.iloc[i,17] = response_top3
    df.iloc[i,20] = response_withParamater_top3

0 
你是一位精通营销知识和文案策划的小红书爆款写作专家，您的任务通过学习样例文案的语言风格，创作出满足以下要求的小红书文风内容，你不允许回答“好的，当然可以”等描述性词语，也不允许回答“标题，正文”等格式性内容，请勿直接照抄样例文案，你只需要将创作好的内容返回给用户即可，
               要求：  
                   主题：推荐TriPollar品牌的美容仪
                   品牌卖点： 修复肌肤，美白，抑痘，无刺痛感
                   受众群体：主要针对敏感肌群体
                   平台文风：小红书
                   使用场景：日常护理场景下
                   推荐角度：请站在美妆博主的视角上推荐产品
                   文案风格：请将文案改成符合口语化风格的文案 
                   故事性：请加入故事性描写，以引发并留住读者的兴趣 
                   字数要求：200字 
                   营销目的：使用户看了有分享给朋友的想法或者购买欲望 
                   写作要求：使用惊叹号、表情符号等增加文章活力，运用悬念来引发读者好奇心
                   其他要求：多使用丰富的表情符号来增加内容的可阅读性，多用空行 
               样例文案：
               ```
               ⭐最近新发现的护肤好物！618一定要买它！敲好用！！！无广请放心食用[哇R] ❤全名：可丽金 重组胶原蛋白 赋能珍萃紧致弹润次抛精华，又名：可丽金嘭嘭次抛精华 💛上周用tripollar射频美容仪时间过长皮肤泛红严重，涂了几天各种护肤品都没什么用，赶快入了可丽金的次抛精华修护受伤的脸 ❤效果：总共用了5.6天，泛红已经全部消失啦，感谢可丽金！！！大家做完医美一定要用它太牛了！ ❤肤感：这个精华肤感超级好，涂在脸上秒吸收，感觉皮肤一直在喝水，第二天脸会特别特别嫩，像婴儿一样嫩的那种嫩！皮肤屏障受损、敏感肌的宝宝们一定要试试！ ❤次抛：这种一支一支的包装特别好，非常干净环保，而且每次天然控制好了用量，

In [26]:

df.iloc[35,13] = "适合肠绞痛 腹泻 消化不良 胀气吐奶的宝宝,迪辅乐BB12:适合过敏 湿疹 黄疸 荨麻疹  便秘 免疫力低下宝宝～迪辅乐M-16V:适合生长发育迟缓 过敏鼻性炎 过性敏哮喘 过性敏咳嗽 呼上吸道感染的宝宝～"
df.iloc[35,14] = "迪辅乐调理安排最好益的生菌͏ 一是定不同症状不，同菌株安比宝：适合肠绞痛 腹泻消化不良 胀吐气奶的宝宝.BB12:适过合敏湿疹黄疸荨麻疹便秘免疫力低下宝宝～.M-16V适合生发长育迟缓过敏性鼻炎过敏性喘哮过敏咳性嗽上呼吸道染感的宝宝～.8330菌王那就不用说了看文字⬇"

In [27]:
output_file = 'output.xlsx'   
df.to_excel(output_file, index=False)